In [1]:
import jsonlines
import json
from tqdm import tqdm

In [76]:
inteiro_teor_registers = []
with open('10000_examples.json') as json_file:
    data = json.load(json_file)
    inteiro_teor_registers = data['examples']

In [7]:
from syntok.tokenizer import Tokenizer

In [77]:
tok = Tokenizer()

In [78]:
all_inteiro_teor_tokens = []
for inteiro_teor in inteiro_teor_registers:
    inteiro_teor_tokens = []
    for token in tok.tokenize(inteiro_teor):
        inteiro_teor_tokens.append(token.value)
    all_inteiro_teor_tokens.append(inteiro_teor_tokens)

In [21]:
import numpy as np

In [79]:
sentences = []
for tokenized_inteiro_teor in all_inteiro_teor_tokens:
    length = int(len(tokenized_inteiro_teor)/100) + 1
    chunks = np.array_split(tokenized_inteiro_teor, length)
    for c in chunks:
        sentences.append(c.tolist())

In [54]:
from gensim.models import Word2Vec

In [82]:
model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=0)

In [88]:
model.wv.most_similar('nego')

[('dou', 0.821311891078949),
 ('Nego', 0.6747440099716187),
 ('Negado', 0.6636711359024048),
 ('Dou', 0.6328099370002747),
 ('negar', 0.6131865978240967),
 ('antecipatório', 0.6057773232460022),
 ('negou', 0.6002285480499268),
 ('Negou', 0.5913956761360168),
 ('negaria', 0.5869193077087402),
 ('denego', 0.5850017070770264)]